In [2]:
import os
import base64
import ipywidgets as widgets
from IPython.display import display, HTML

# -----------------------------------------------------------
# Inject custom CSS to force file explorer button text to align left.
# This CSS targets both the button element and the inner label.
# The rules below:
#   - Force the button to display as a block element with 100% width.
#   - Ensure text wrapping is enabled.
#   - Set text alignment to left for both the button and its label.
# -----------------------------------------------------------
HTML("""
<style>
.file-explorer-button button,
.file-explorer-button .widget-button-label {
    display: block !important;
    width: 100% !important;
    white-space: normal !important;
    text-align: left !important;
    overflow-wrap: break-word !important;
    word-wrap: break-word !important;
    margin: 0 !important;
    padding-left: 4px !important;
}
</style>
""")

#####################################################
# Global Variables & Initialization
#####################################################

# Start browsing at the current working directory.
current_dir = "/raid/s3/opengptx/user/richard-rutmann/data/eurolingua/experiments/multilinguality/experiments/comparison/ir_summary/"
# List to store full paths of PNG files that have been added.
accumulated_paths = []

# Container for the file browser UI (will be updated dynamically)
file_browser_box = widgets.VBox()

# Output widget for displaying the final grid of images.
output_display = widgets.Output()

#####################################################
# Helper: Create a zoomable image widget
#####################################################

def create_zoomable_image_widget(path):
    """
    Given a file path, return a VBox widget with an image displayed
    in a 400x400 container and a slider to zoom in/out.
    
    The slider's description is the file's basename with the prefix 
    "confusion_matrix_edu_" removed.
    """
    container_width = 350
    container_height = 350
    initial_scale = 1.0

    # Read and encode the image in base64.
    with open(path, 'rb') as f:
        img_data = f.read()
    b64_str = base64.b64encode(img_data).decode('utf-8')

    # Build the HTML display for the image.
    html_template = f"""
    <div style="
         overflow: auto; 
         width: {container_width}px; 
         height: {container_height}px; 
         border: 1px solid #ccc; 
         position: relative;">
      <img src="data:image/png;base64,{b64_str}" 
           style="transform-origin: top left; transform: scale({initial_scale});" />
    </div>
    """
    html_widget = widgets.HTML(value=html_template)

    # Remove the unwanted prefix from the filename.
    file_label = os.path.basename(path).replace("confusion_matrix_edu_", "")

    # Create a zoom slider with the modified label.
    slider = widgets.FloatSlider(
        value=initial_scale,
        min=0.5,
        max=4.0,
        step=0.1,
        description=file_label,
        continuous_update=True,
        layout=widgets.Layout(width='90%')
    )

    # Update the HTML display when the slider value changes.
    def on_slider_change(change):
        scale = change['new']
        updated_html = f"""
        <div style="
             overflow: auto; 
             width: {container_width}px; 
             height: {container_height}px; 
             border: 1px solid #ccc; 
             position: relative;">
          <img src="data:image/png;base64,{b64_str}" 
               style="transform-origin: top left; transform: scale({scale});" />
        </div>
        """
        html_widget.value = updated_html

    slider.observe(on_slider_change, names='value')
    return widgets.VBox([html_widget, slider])

#####################################################
# File Browser UI
#####################################################

def update_file_browser():
    """
    Refresh the file browser: display the current directory's
    subdirectories and PNG files as clickable buttons.
    
    For PNG file buttons, the displayed name is modified to remove 
    the prefix "confusion_matrix_edu_". All buttons are styled with
    the "file-explorer-button" class to force left alignment.
    """
    global current_dir, accumulated_paths

    # Header: display current directory and a Back button.
    current_dir_label = widgets.HTML(value=f"<b>Current Directory:</b> {current_dir}")
    
    # Determine the parent directory for the Back button.
    parent_dir = os.path.dirname(current_dir)
    back_button = widgets.Button(description="← Back", button_style="info")
    back_button.disabled = (parent_dir == current_dir)
    
    def on_back_clicked(b):
        global current_dir
        current_dir = parent_dir
        update_file_browser()
    back_button.on_click(on_back_clicked)
    
    header = widgets.HBox([back_button, current_dir_label])
    
    # List items in the current directory.
    try:
        entries = os.listdir(current_dir)
    except Exception as e:
        entries = []
    
    # Create a list to hold buttons for directories and PNG files.
    buttons = []
    
    # Sort entries so that directories come first.
    entries.sort()
    for entry in entries:
        full_path = os.path.join(current_dir, entry)
        if os.path.isdir(full_path):
            # Directory button (with a folder emoji).
            btn = widgets.Button(
                description=f"📁 {entry}",
                layout=widgets.Layout(width="auto"),
                button_style=""
            )
            # Add our custom class for left alignment.
            btn.add_class("file-explorer-button")
            # Clicking a directory enters it.
            def on_dir_clicked(b, entry=entry):
                global current_dir
                current_dir = os.path.join(current_dir, entry)
                update_file_browser()
            btn.on_click(on_dir_clicked)
            buttons.append(btn)
        elif os.path.isfile(full_path) and entry.lower().endswith('.png'):
            # PNG file button (with a picture emoji).
            # Remove the prefix "confusion_matrix_edu_" from the file name.
            short_entry = entry.replace("confusion_matrix_edu_", "")
            already_added = full_path in accumulated_paths
            # Set a fixed width (250px as updated) so that the text wraps if it is long.
            btn = widgets.Button(
                description=f"🖼️ {short_entry}",
                layout=widgets.Layout(width="250px"),
                button_style="success" if not already_added else "warning",
                disabled=already_added
            )
            # Add our custom class to force left alignment.
            btn.add_class("file-explorer-button")
            # Clicking a PNG file adds it to the accumulated list.
            def on_file_clicked(b, full_path=full_path):
                global accumulated_paths
                if full_path not in accumulated_paths:
                    accumulated_paths.append(full_path)
                update_remove_dropdown()
                update_file_browser()  # refresh to disable the button
            btn.on_click(on_file_clicked)
            buttons.append(btn)
    
    # Arrange buttons in a grid (4 columns).
    grid = widgets.GridBox(
        buttons,
        layout=widgets.Layout(
            grid_template_columns="repeat(4, 1fr)",
            grid_gap="5px"
        )
    )
    
    # Update the file_browser_box container.
    file_browser_box.children = [header, grid]

#####################################################
# Remove Accumulated Files UI
#####################################################

# Dropdown to list accumulated files (by basename)
remove_dropdown = widgets.Dropdown(
    description="Remove:",
    options=[]
)

remove_button = widgets.Button(
    description="Remove",
    button_style="danger"
)

def update_remove_dropdown():
    """Update the remove dropdown with the basenames of the accumulated files."""
    global accumulated_paths
    remove_dropdown.options = [os.path.basename(p) for p in accumulated_paths]

def on_remove_clicked(b):
    global accumulated_paths
    selected = remove_dropdown.value
    if not selected:
        return
    # Remove the file whose basename matches (assumes unique names).
    for p in accumulated_paths:
        if os.path.basename(p) == selected:
            accumulated_paths.remove(p)
            break
    update_remove_dropdown()
    update_file_browser()  # Refresh file browser so file buttons re-enable

remove_button.on_click(on_remove_clicked)
remove_ui = widgets.HBox([remove_dropdown, remove_button])

#####################################################
# Display Images UI
#####################################################

def chunk_list(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

display_button = widgets.Button(
    description="Display All/ Refresh View",
    button_style="success"
)

def on_display_clicked(b):
    output_display.clear_output()  # Clear previous display
    if not accumulated_paths:
        with output_display:
            print("No images have been added yet.")
        return

    max_images = 20  # Limit display to at most 20 images
    paths_to_show = accumulated_paths[:max_images]
    
    # Create a widget for each image file.
    image_widgets = [create_zoomable_image_widget(p) for p in paths_to_show]
    
    # Arrange widgets in rows (4 per row).
    rows = [widgets.HBox(list(chunk)) for chunk in chunk_list(image_widgets, 4)]
    
    with output_display:
        display(widgets.VBox(rows))

display_button.on_click(on_display_clicked)

#####################################################
# Assemble the Final UI
#####################################################

# Initialize the file browser for the first time.
update_file_browser()

# Assemble the UI components.
ui_layout = widgets.VBox([
    widgets.HTML("<h3>Custom File Explorer</h3>"),
    file_browser_box,
    widgets.HTML("<hr>"),
    remove_ui,
    widgets.HTML("<hr>"),
    display_button,
    output_display
])

display(ui_layout)
